In [1]:
import pandas as pd 
import string
import numpy as np
import sklearn as sk
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from transformers import AutoTokenizer, TFBertModel 

C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\Users\LENOVO\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
cwd = os.getcwd()
datapath = os.path.join(cwd, 'dataset')

In [3]:
df = pd.read_csv(os.path.join(datapath, 'df_processed.csv'))
df['berita'] = df['berita'].apply(lambda x : " ".join(eval(x)))
# df['berita'] = df['berita'].apply(lambda x : eval(x))

df.head()

,Unnamed: 0,berita,label
0,0,gunung agung erupsi pertama kali november letu...,0
1,1,jakarta cnn indonesia menteri bumn erick thohi...,0
2,2,dosen fakultas dokter hewan ipb yusuf ridwan n...,0
3,3,jakarta anggota tni serda n serda da tusuk ora...,0
4,4,aku tembak jatuh pesawat ukraina iran tuai gel...,0


In [4]:
X, y = df['berita'], df['label']
X.shape, y.shape

((2198,), (2198,))

In [5]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42,
                                     stratify=df['label'])

In [6]:
df_train.shape, df_test.shape

((1538, 3), (660, 3))

In [7]:
model = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model)
bert = TFBertModel.from_pretrained(model)

max_len = 70

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [8]:
X_train = tokenizer(
    text=df_train['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

X_test = tokenizer(
    text=df_test['berita'].tolist(),
    add_special_tokens=True,
    max_length=max_len,
    truncation=True,
    padding=True,
    return_tensors='tf',
    return_token_type_ids=False,
    return_attention_mask=True,
    verbose=True
)

In [9]:
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
# embeddings = dbert_model(input_ids, attention_mask = input_mask)[0]

embeddings = bert(input_ids, attention_mask = input_mask)[0] # 0 = last hidden state, 1 = poller_output
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32, activation='relu')(out)

y = Dense(2)(out)

model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # HF recommendation
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0
)

loss = CategoricalCrossentropy(from_logits=True)
metric = CategoricalAccuracy('balanced_accuracy')

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metric
)

history = model.fit(
    x = {'input_ids':X_train['input_ids'], 'attention_mask':X_train['attention_mask']},
    y = to_categorical(df_train['label']),
    validation_data = ({'input_ids':X_test['input_ids'], 'attention_mask':X_test['attention_mask']},
                        to_categorical(df_test['label'])),
    epochs=1,
    batch_size=32
)

49/49 [==============================] - 3164s 64s/step - loss: 0.6007 - balanced_accuracy: 0.6385 - val_loss: 0.5074 - val_balanced_accuracy: 0.7152


In [10]:
from sklearn.metrics import classification_report

predicted = model.predict({'input_ids': X_test['input_ids'], 'attention_mask': X_test['attention_mask']})
y_predicted = np.argmax(predicted, axis=1)


21/21 [==============================] - 352s 17s/step


KeyError: 'Sentiment'

In [11]:
print(classification_report(df_test['label'], y_predicted))

              precision    recall  f1-score   support

           0       0.89      0.60      0.71       391
           1       0.60      0.89      0.72       269

    accuracy                           0.72       660
   macro avg       0.74      0.74      0.72       660
weighted avg       0.77      0.72      0.71       660

